# Verify the stage bias in MetaWorld

In [ ]:
import os
import numpy as np
import pandas as pd
import random
from IPython.display import display
import time
import platform
import subprocess
import pickle

# setting
env_name = 'drawer-open'  # ['door-open', 'drawer-open', 'window-open']
long_env_name = 'metaworld_' + env_name + '-v2'

In [ ]:
def load_segments(output_dir):
    segments = []
    for filename in os.listdir(output_dir):
        if not filename.endswith(".pkl"):
            continue
        
        pkl_path = os.path.join(output_dir, filename)
        video_filename = filename.replace('.pkl', '.gif')
        video_path = os.path.join(output_dir, video_filename)
        
        if not os.path.exists(video_path):
            print(f"warning: video {video_filename} does not exist, the segment is skipped")
            continue
        
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            start_idx = data['start_step']
            episode = data['source_traj']
            reward_sum = sum(data['rewards'])
            
            segments.append({
                'start_idx': start_idx,
                'episode': episode,
                'reward_sum': reward_sum,
                'video_path': video_path
            })
    return segments


In [ ]:
base_path = f"./segment/{long_env_name}/"
segments = load_segments(base_path)

print(f'segment_len = {len(segments)}')

output_csv = os.path.join(base_path, f"human.csv")
if os.path.exists(output_csv):
    df = pd.read_csv(output_csv)
else:
    df = pd.DataFrame(columns=[
        'segment1_start_idx', 'segment1_episode', 'segment1_reward_sum',
        'segment2_start_idx', 'segment2_episode', 'segment2_reward_sum',
        'human_preference'
    ])

## Prompt

### door-open

The target behavior is that the robot arm smoothly rotates the door until it stays fully open at a clearly visible angle.
If the arm moves abnormally, lower your priority for the segment.

### drawer-open
The target behavior is that the drawer is fully extended to its final position with controlled, direct pushing.
If the arm moves abnormally, lower your priority for the segment.

### window-open
The target behavior is that the window slides horizontally to a clearly open position with coordinated gripper guidance.
If the arm moves abnormally, lower your priority for the segment.

In [ ]:
from IPython.display import clear_output, HTML, Image
import base64

for i in range(20):
    clear_output()
    print(f"{i+1}th among total 20 feedbacks")
    seg1, seg2 = random.sample(segments, 2)

    time.sleep(0.1)
    display(HTML(f'''
    <div style="display: inline-block; margin-right: 100px;">
        <p> Please enter '1' for prefer the left segment or '2' for the right or '0' for skip. </p>
        <img src="{seg1['video_path']}" width="400" loop="true" >
        <img src="{seg2['video_path']}" width="400" loop="true" >
    </div>
    '''))

    PERF_MAPPING = {
        "1": "1 0",
        "2": "0 1",
        "0": "- -"
    }
    time.sleep(1)
    select = input("Please enter '1' for prefer the left segment or '2' for the right or '0' for skip.1")
    if select == 'quit' or select == 'exit':
        break
    elif select not in ['1', '2', '0']:
        print("Invalid input. ")
        continue
    new_entry = {
        'segment1_start_idx': seg1['start_idx'],
        'segment1_episode': seg1['episode'],
        'segment1_reward_sum': f"{seg1['reward_sum']:.2f}",
        'segment2_start_idx': seg2['start_idx'],
        'segment2_episode': seg2['episode'],
        'segment2_reward_sum': f"{seg2['reward_sum']:.2f}",
        'human_preference': PERF_MAPPING[select]
    }
    df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)

    df.to_csv(output_csv, index=False)
    print(f"\nsave to {output_csv}")


df.to_csv(output_csv, index=False)
print(f"\nsave to {output_csv}")